## Q1

In [45]:
# import pandas as pd
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)

In [46]:
df = pd.read_csv('AB_test_data.csv')

In [47]:
df.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   purchase_TF  130000 non-null  bool  
 1   Variant      130000 non-null  object
 2   date         130000 non-null  object
 3   id           130000 non-null  object
dtypes: bool(1), object(3)
memory usage: 3.1+ MB


In [49]:
P_a=float(df.query('Variant=="A"').query('purchase_TF==True')['id'].count())/float(df.query('Variant=="A"')['id'].count())

In [50]:
P_b=float(df.query('Variant=="B"').query('purchase_TF==True')['id'].count())/float(df.query('Variant=="B"')['id'].count())

In [51]:
P_a

0.149616

In [52]:
P_b

0.1766

In [53]:
from statsmodels.stats.proportion import proportions_ztest

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   purchase_TF  130000 non-null  bool  
 1   Variant      130000 non-null  object
 2   date         130000 non-null  object
 3   id           130000 non-null  object
dtypes: bool(1), object(3)
memory usage: 3.1+ MB


In [55]:
control_results = df[(df['purchase_TF'] == True) &(df['Variant']=='A')]['purchase_TF']
control_results

10        True
19        True
22        True
37        True
45        True
          ... 
129978    True
129980    True
129982    True
129988    True
129989    True
Name: purchase_TF, Length: 18702, dtype: bool

In [56]:
treatment_results = df[(df['purchase_TF'] == True)&(df['Variant']=='B')]['purchase_TF']
treatment_results

181       True
415       True
429       True
553       True
726       True
          ... 
128735    True
128881    True
129016    True
129048    True
129798    True
Name: purchase_TF, Length: 883, dtype: bool

In [57]:
n_treat = df[df['Variant']=='B']['id'].count()

In [58]:
n_treat

5000

In [59]:
n_con = df[df['Variant']=='A']['id'].count()
n_treat = df[df['Variant']=='B']['id'].count()
successes = [treatment_results.sum(),control_results.sum() ]
nobs = [n_treat, n_con]

In [60]:
n_con

125000

In [61]:
successes

[883, 18702]

In [62]:
z_stat, pval = proportions_ztest(successes, nobs=nobs)

In [63]:
z_stat

5.2304877183522045

### Since 5.23>1.64, the null hypothesis is rejected and the new interface does increase the conversion rate.

## Q2

In [64]:
#optimal_n=((1.96*np.sqrt(2*p_bar*(1-p_bar))+0.2*np.sqrt(P_b*(1-P_b)+P_a*(1-P_a)))**2)/((0.02)**2)

In [65]:
#optimal=optimal_n.round()

In [66]:
#Geting a single sample to find the z-score
treatment=df[df['Variant']=='B']

control=df[df['Variant']=='A']

from random import sample

control1=control.sample(n=3184)

treatment1=treatment.sample(n=3184)

treatment_results1= treatment1[treatment1['purchase_TF'] == True]['purchase_TF']
control_results1= control1[control1['purchase_TF'] == True]['purchase_TF']

successes1 = [control_results1.sum(), treatment_results1.sum()]

z_stat1, pval1 = proportions_ztest(successes1, nobs=[3184,3184])

z_stat1

-3.463193447083688

In [67]:
pval1

0.0005338044698854194

In [68]:
P_a= control_results1.sum()/3184
P_a

0.1466708542713568

In [69]:
P_b= treatment_results1.sum()/3184
P_b

0.17870603015075376

In [70]:
#Find the 10 random samples
random_num_dict = {}
for i in range(10):
    random_num_dict[i] = [control.sample(n=3184),treatment.sample(n=3184)]
#print(random_num_dict)

z_stats_list = []
p_val_list = []
P_a_list = []
P_b_list = []

for key in random_num_dict.keys():
    #obtaining the results(randomized control/treatment group) by key
    df_control_result = random_num_dict[key][0]
    control_results = df_control_result[control['purchase_TF'] == True]['purchase_TF']
    df_treatment_result = random_num_dict[key][1]
    treatment_results = df_treatment_result[treatment['purchase_TF'] == True]['purchase_TF']
    
    successes = [treatment_results.sum(),control_results.sum()]
    
    z_stat, p_val = proportions_ztest(successes, nobs=[3184,3184])
    z_stats_list.append(z_stat)
    p_val_list.append(p_val)
    
    P_a = control_results.sum()/3184
    P_b = treatment_results.sum()/3184
    
    P_a_list.append(P_a)
    P_b_list.append(P_b)
    
    
print(z_stats_list)
print(p_val_list)
print(P_a_list)
print(P_b_list)
#each list is indiced according to range(10), e.g. z_stats_1 = z_stats_list[0], p_val_1 = p_val_list[0].

[1.244288294783038, 3.331467821103587, 2.689620280708866, 3.2944409064613525, 3.901229255472357, 3.56643948782784, 1.5654941965031337, 2.0435515233687194, 4.811815517820096, 1.8510147271896575]
[0.21339347748672044, 0.000863892871976199, 0.007153336302187226, 0.0009861767780147157, 9.570546034882321e-05, 0.0003618643189814297, 0.11746707710455727, 0.04099787244644125, 1.4956538220375277e-06, 0.06416743443140618]
[0.16080402010050251, 0.14290201005025124, 0.14918341708542712, 0.1513819095477387, 0.1416457286432161, 0.14604271356783918, 0.15452261306532664, 0.15201005025125627, 0.13850502512562815, 0.15766331658291458]
[0.17242462311557788, 0.17336683417085427, 0.17399497487437185, 0.18216080402010051, 0.1774497487437186, 0.17902010050251257, 0.16896984924623115, 0.1708542713567839, 0.1827889447236181, 0.17493718592964824]


/var/folders/fc/f8fdl3p13ml6fcdkp0kht73h0000gp/T/ipykernel_41603/688659106.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_results = df_control_result[control['purchase_TF'] == True]['purchase_TF']
/var/folders/fc/f8fdl3p13ml6fcdkp0kht73h0000gp/T/ipykernel_41603/688659106.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treatment_results = df_treatment_result[treatment['purchase_TF'] == True]['purchase_TF']


In [71]:
results = pd.DataFrame(list(zip(z_stats_list, P_a_list,P_b_list)),
               columns =['Z_score', 'P_a','P_b'])
results
    

,Z_score,P_a,P_b
0,1.244288,0.160804,0.172425
1,3.331468,0.142902,0.173367
2,2.689620,0.149183,0.173995
3,3.294441,0.151382,0.182161
4,3.901229,0.141646,0.177450
5,3.566439,0.146043,0.179020
6,1.565494,0.154523,0.168970
7,2.043552,0.152010,0.170854
8,4.811816,0.138505,0.182789
9,1.851015,0.157663,0.174937


In [72]:
for i in range(1,len(z_stats_list)+1):
    if z_stats_list[i-1] > 1.64:
        print("In trial %d, the null hypothesis is rejected. B increases the conversion rate." % i)
    else:
        print("In trial %d, the null hypothesis cannot be rejected with sufficient evidence." % i)

In trial 1, the null hypothesis cannot be rejected with sufficient evidence.
In trial 2, the null hypothesis is rejected. B increases the conversion rate.
In trial 3, the null hypothesis is rejected. B increases the conversion rate.
In trial 4, the null hypothesis is rejected. B increases the conversion rate.
In trial 5, the null hypothesis is rejected. B increases the conversion rate.
In trial 6, the null hypothesis is rejected. B increases the conversion rate.
In trial 7, the null hypothesis cannot be rejected with sufficient evidence.
In trial 8, the null hypothesis is rejected. B increases the conversion rate.
In trial 9, the null hypothesis is rejected. B increases the conversion rate.
In trial 10, the null hypothesis is rejected. B increases the conversion rate.


## Q3

In [76]:
A = np.log((1-0.2)/0.05)
B = np.log(0.2/(1-0.05)) #Wald Boundaries
print(A,B)

2.772588722239781 -1.5581446180465497


In [74]:
num_of_iter = []

for k in range(10):#10 samples
    temp = 0
    count = 0
    sample_control = df[df['Variant'] == 'A']
    sample_treatment = df[df['Variant'] == 'B'].sample(n=3184, random_state = k)
    
    
    ab_test = pd.concat([sample_control, sample_treatment], axis=0)
    ab_test.reset_index(drop=True, inplace=True)
    
    control_res = ab_test[ab_test['Variant'] == 'A']['purchase_TF']
    treatment_res = ab_test[ab_test['Variant'] == 'B']['purchase_TF']
    #whether the customer buy anything or not
    
    conv_A = control_res.sum() / control_res.count()
    conv_B = treatment_res.sum() /treatment_res.count()
    #the conversion rate
    
    new_idx = sample_treatment.reset_index()
    
    X_1 = np.log(conv_B/conv_A)
    X_0 = np.log((1-conv_B)/(1-conv_A))
    
    for l in range(1,3185): #within the range of the "optimal sample size"
        if B < temp < A:
            if new_idx['purchase_TF'][l-1] == 1:
                temp += X_1
            if new_idx['purchase_TF'][l-1] == 0:
                temp += X_0
            count = l
        else:
            break
    num_of_iter.append(count)

print(num_of_iter)
    
    

[317, 568, 824, 686, 493, 553, 814, 320, 260, 444]


In [77]:
print("The average iteration time is: "+str(sum(num_of_iter)/len(num_of_iter)))

The average iteration time is: 527.9
